In [1]:
from config import settings
import logging
import traceback

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(name)s] [%(levelname)s] %(message)s",
    handlers=[logging.FileHandler("debug.log"), logging.StreamHandler()],
)

In [2]:
import aiofiles
import json


async def save_to_json(data, params):
    logging.debug(f"Saving data to {params['filename']}")
    async with aiofiles.open(params["filename"], "w") as f:
        await f.write(json.dumps(data))

In [3]:
async def print_data(data, params):
    logging.debug(f"Print data: {params['name']}")
    print(f"{params['name']} : {json.dumps(data)}")

In [4]:
from motor.motor_asyncio import AsyncIOMotorClient


class DatabaseClient:
    def __init__(self, config):
        self.client = AsyncIOMotorClient(config.db.uri)
        self.db = self.client[config.db.name]

    async def save_to_db(self, data, collection_name):
        try:
            collection = self.db[collection_name]
            result = await collection.insert_one(data)
            return result.inserted_id
        except Exception as e:
            logging.error(f"Error saving data to database: {e}")


STORAGE_METHOD = None
DB_CLIENT = None

if settings.storage_method == "db":
    DB_CLIENT = DatabaseClient(settings)


async def save_to_db(data, params):
    client = AsyncIOMotorClient(db_uri)
    db = client[db_name]
    collection = db[collection_name]
    try:
        result = await collection.insert_one(data)
        return result.inserted_id
    except Exception as e:
        print(f"Error saving data to database: {e}")
    finally:
        await client.close()

In [5]:
async def store_data(data, storage_method, params):
    logging.debug(f"Store data: {params['name']}")
    await storage_method(data, params)

In [6]:
def format_ob_data(exchange, data):
    return {"exchange": exchange, "ob": data}


def format_funding_data(exchange, data):
    return {
        "exchange": exchange,
        "symbol": data["symbol"],
        "mark_price": data["markPrice"],
        "index_price": data["indexPrice"],
        "funding": data["fundingRate"],
        "next_funding": data["nextFundingRate"],
        "timestamp": data["timestamp"],
        "funding_timestamp": data["fundingTimestamp"],
    }


def format_oi_data(exchange, data):
    return {
        "exchange": exchange,
        "oi": str(data["open_interest"]),
        "symbol": data["symbol"],
        "timestamp": data["timestamp"],
    }

In [7]:
async def save_oi_data(exchange, data):
    logging.debug(f"Saving oi from {exchange}")
    await store_data(format_oi_data(exchange, data), print_data, {"name": "oi"})


async def save_ob_data(exchange, data):
    logging.debug(f"Saving ob from {exchange}")
    await store_data(format_ob_data(exchange, data), print_data, {"name": "ob"})


async def save_funding_data(exchange, data):
    logging.debug(f"Saving funding from {exchange}")
    await store_data(
        format_funding_data(exchange, data), print_data, {"name": "funding"}
    )

In [8]:
from cryptofeed import FeedHandler
from cryptofeed.defines import OPEN_INTEREST


async def oi_handler(data, receipt):
    try:
        await save_oi_data(data.exchange, data.to_dict())
    except Exception as e:
        logging.error(f"OI handler error: {e}")


def add_exchange_feed(fh, exchange_id, symbols):
    try:
        match exchange_id:
            case "BinanceFutures":
                from cryptofeed.exchanges import BinanceFutures

                fh.add_feed(
                    BinanceFutures(
                        symbols=symbols,
                        channels=[OPEN_INTEREST],
                        callbacks={OPEN_INTEREST: oi_handler},
                    )
                )
            case "Bitmex":
                from cryptofeed.exchanges import Bitmex

                fh.add_feed(
                    Bitmex(
                        symbols=symbols,
                        channels=[OPEN_INTEREST],
                        callbacks={OPEN_INTEREST: oi_handler},
                    )
                )
            case "OKX":
                from cryptofeed.exchanges import OKX

                fh.add_feed(
                    OKX(
                        symbols=symbols,
                        channels=[OPEN_INTEREST],
                        callbacks={OPEN_INTEREST: oi_handler},
                    )
                )
            case _:
                raise RuntimeError(f"Unsupported exchange")
    except Exception as e:
        logging.error(f"Error adding {exchange_id} feed: {e}")


async def run_cryptofeed(config):
    fh = FeedHandler()
    for ex in config.cryptofeed.exchanges:
        add_exchange_feed(fh, ex, config.cryptofeed.oi_symbols)

    fh.run(start_loop=False)


# loop = asyncio.get_event_loop()
# loop.create_task(aio_task())
# loop.run_forever()

In [9]:
import ccxt.async_support as ccxt
import asyncio
from datetime import datetime


async def fetch_order_book(exchange, symbol):
    logging.debug(f"Fetching order book for {symbol} from {exchange.id}")
    try:
        order_book = await exchange.fetch_l2_order_book(symbol)
        # Check if 'timestamp' is present in the order book data
        if "timestamp" not in order_book:
            # Append the current UTC timestamp
            order_book["timestamp"] = (
                datetime.utcnow().timestamp() * 1000
            )  # Convert to milliseconds
        return order_book
    except Exception as e:
        logging.error(f"Error fetching order book for {symbol}: {e}")


async def fetch_funding_rate(exchange, symbol):
    logging.debug(f"Fetching funding rate for {symbol} from {exchange.id}")
    try:
        funding_rate = await exchange.fetch_funding_rate(symbol)
        return funding_rate
    except Exception as e:
        logging.error(f"Error fetching funding rate for {symbol}: {e}")

In [10]:
async def fetch_and_save_funding(exchange, symbol):
    data = await fetch_funding_rate(exchange, symbol)
    await save_funding_data(exchange.id, data)


async def fetch_and_save_ob(exchange, symbol):
    data = await fetch_order_book(exchange, symbol)
    await save_ob_data(exchange.id, data)

In [11]:
import time


def instantiate_exchanges(config):
    exchanges = []
    logging.info("Initializing exchange connections...")
    for e in config.ccxt.exchanges:
        logging.info(f"Connecting to {e.id}")
        exchange_class = getattr(ccxt, e.id)
        exchanges.append(
            {
                "exchange": exchange_class({"enableRateLimit": True, "verbose": False}),
                "ob_symbols": e.ob_symbols,
                "funding_symbols": e.funding_symbols,
            }
        )
        if e.id == "coinbase":
            logging.debug("Using authenticated coinbase connection")
            exchanges[-1]["exchange"].apiKey = config.coinbase.api_key
            exchanges[-1]["exchange"].secret = config.coinbase.api_secret
    return exchanges


def terminate_connections(exchanges):
    for e in exchanges:
        e["exchange"].close()


async def fetch_data(data_sources):
    while True:
        logging.info("Fetching data...")
        start = time.perf_counter()
        tasks = []
        for source in data_sources:
            exchange = source["exchange"]
            try:
                for symbol in source["ob_symbols"]:
                    task = asyncio.create_task(fetch_and_save_ob(exchange, symbol))
                    tasks.append(task)

                for symbol in source["funding_symbols"]:
                    task = asyncio.create_task(fetch_and_save_funding(exchange, symbol))
                    tasks.append(task)

            except Exception as e:
                tb_str = traceback.format_exc()
                logging.error(f"Error fetching data from {exchange.id}: {e}")
                logging.debug(tb_str)

        await asyncio.gather(*tasks)
        elapsed = time.perf_counter() - start
        logging.info(f"Data fetched in {elapsed:0.5f} seconds.")
        await asyncio.sleep(max(60 - elapsed, 0))

In [12]:
async def main():
    logging.info("Recording started.")
    exchanges = []
    try:
        exchanges = instantiate_exchanges(settings)
        fetch_task = asyncio.create_task(fetch_data(exchanges))
        feed_task = asyncio.create_task(run_cryptofeed(settings))
        await asyncio.gather(fetch_task, feed_task)
    except Exception as e:
        tb_str = traceback.format_exc()
        logging.critical(f"Critical error: {e}. {tb_str}")

In [13]:
# await main()

In [14]:
from config import mappings


def map_data(data, format, exchange, channel):
    if exchange == "bitfinex":
        print(data)
    out = {"exchange": exchange, "channel": channel}
    if not "timestamp" in format.mapping:
        out["ts"] = datetime.utcnow().timestamp() * 1000
    for k, v in format.mapping.items():
        out[k] = data[v]
    return out


def format_exchange_data(exchange, channel, message):
    results = []
    try:
        format = mappings[exchange][channel]
        data = message[format.data_root] if format.data_root else message
        if exchange == "bitfinex":
            print(message)
            print(data)
        if format.is_array:
            items = data[format.array_root] if format.array_root else data
            for item in items:
                results.append(map_data(item, format, exchange, channel))
        else:
            results.append(map_data(data, format, exchange, channel))
    except Exception as e:
        logging.error(f"Error mapping data for {exchange} {channel}: {e}")
    return results


async def save_exchange_data(exchange, channel, data):
    if data is not None:
        logging.debug(f"Saving data from {exchange}")
        await store_data(
            format_exchange_data(exchange, channel, data),
            print_data,
            {"name": "ticker"},
        )
    else:
        logging.warning(f"No data to save from {exchange}")


async def save_tvl_data(protocol, data):
    if data is not None:
        logging.debug(f"Saving tvl data from {protocol}")
        await store_data(
            {
                "ts": datetime.utcnow().timestamp() * 1000,
                "protocol": protocol,
                "tvl": data,
            },
            print_data,
            {"name": "tvl"},
        )
    else:
        logging.warning(f"No data to save from {protocol}")


async def fetch_from_url(session, url):
    try:
        async with session.get(url) as response:
            if response.status == 200:
                data = await response.json()
                return data
            else:
                logging.error(f"Error fetching from {url}: {response.status}, {data}")
                return None
    except Exception as e:
        logging.error(f"Exception during fetching from {url}: {e}")
        logging.error(traceback.format_exc())


async def fetch_and_save_http(session, exchange, url, channel, interval):
    while True:
        data = await fetch_from_url(session, url)
        await save_exchange_data(exchange, channel, data)
        await asyncio.sleep(interval / 1000)  # Convert milliseconds to seconds


async def fetch_and_save_tvl(session, protocol, url, interval):
    while True:
        data = await fetch_from_url(session, url + protocol)
        await save_tvl_data(protocol, data)
        await asyncio.sleep(interval / 1000)

In [15]:
import aiohttp
import asyncio
import logging


async def fetch_http_data(config):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for endpoint in settings.http.endpoints:
            task = asyncio.create_task(
                fetch_and_save_http(
                    session,
                    endpoint.exchange,
                    endpoint.url,
                    endpoint.channel,
                    endpoint.interval,
                )
            )
            tasks.append(task)
        for protocol in settings.defillama.protocols:
            task = asyncio.create_task(
                fetch_and_save_tvl(
                    session,
                    protocol,
                    settings.defillama.endpoint,
                    settings.defillama.interval,
                )
            )
            tasks.append(task)
        await asyncio.gather(*tasks)

In [16]:
from config import settings
import traceback


async def main():
    logging.info("Recording started.")
    try:
        http_task = asyncio.create_task(fetch_http_data(settings))
        await asyncio.gather(http_task)
    except Exception as e:
        tb_str = traceback.format_exc()
        logging.critical(f"Critical error: {e}. {tb_str}")

In [ ]:
await main()

In [ ]:
import json
import asyncio
import websockets

async def websocket_client(url, topic, update_interval, heartbeat_interval):
    last_processed_time = 0
    last_heartbeat_time = 0

    async with websockets.connect(url) as ws:
        # Subscribe to the topic
        sub_message = json.dumps({"op": "subscribe", "args": [topic]})
        await ws.send(sub_message)

        while True:
            current_time = time.time()

            # Send heartbeat message at the required interval
            if current_time - last_heartbeat_time >= heartbeat_interval:
                heartbeat_message = json.dumps({"op": "ping"})
                await ws.send(heartbeat_message)
                last_heartbeat_time = current_time

            try:
                # Receive and process messages
                message = await ws.recv()
                if current_time - last_processed_time >= update_interval:
                    data = json.loads(message)
                    # Process the received data
                    last_processed_time = current_time
                # Else, skip this message
            except websockets.exceptions.ConnectionClosed:
                break
            except Exception as e:
                # Handle exceptions


In [ ]:
async def main():
    ws_url = "wss://stream.bybit.com/v5/public/linear"
    topic = "instrument_info.100ms.BTCUSD"  # Example topic
    update_interval = 2  # Rate limit to one update every 2 seconds
    heartbeat_interval = 30  # Send a heartbeat every 30 seconds
    task = asyncio.create_task(
        websocket_client(ws_url, topic, update_interval, heartbeat_interval)
    )
    await asyncio.gather(task)